In [1]:
import re
import numpy as np
import pandas as pd
from unidecode import unidecode

In [3]:
entity = pd.read_csv('entity_210430.csv',encoding = 'utf-8')
entity = entity.drop('Unnamed: 0', axis = 1)
entity

,entityName,entityId:ID,entityProperty,entityType,entityImportance,:LABEL
0,"Iraqi parliamentary election, September 1954",0,"title: Iraqi parliamentary election, Septemb...",Election,8.734346e+00,Event
1,New Road Team,1,"name: New Road, name: New Road Team, nick: N...",SoccerClub,1.066392e+01,Organisation
2,Here Comes Trouble (Bad Company album),2,"name: Here Comes Trouble, releaseDate: 1992-0...",Album,2.728453e+03,Work
3,Canna glauca,3,"synonym: * Canna angustifolia (L.), synonym: ...",Plant,1.519769e+00,Species
4,Povla Frijsh,4,"name: Povla Frijsh, surname: Frijsh, givenNa...",Person,1.612313e-01,Person
...,...,...,...,...,...,...
4691086,German submarine U-133 (1941),5822928,"name: U-133, orderDate: 1939-08-07, layingDo...",Ship,9.038188e-01,Others
4691087,Rengarajan Jaiprakash,5822929,"name: Jaiprakash Rengarajan, description: Ind...",Person,9.778662e+01,Person
4691088,Jean d'Eaubonne,5822930,"name: Jean d Eaubonne, surname: Eaubonne, gi...",Person,1.612313e-01,Person
4691089,"Jack Price (footballer, born 1877) 2",5822931,"years: 1897, numberOfMatches: 1, numberOfGoa...",CareerStation,6.397571e-01,TimePeriod


In [6]:
relationship = pd.read_csv('relationship_201203.csv',encoding = 'utf-8')
relationship = relationship.drop('Unnamed: 0', axis = 1)
relationship

,:START_ID,:TYPE,:END_ID,inverse
0,362633,birthPlace,5533231,direct
1,362633,birthPlace,1056076,direct
2,362633,deathPlace,5348598,direct
3,362633,deathPlace,3773633,direct
4,1324976,birthPlace,3429596,direct
...,...,...,...,...
26377579,1862626,type,4326933,directed
26377580,1862626,type,193423,directed
26377581,5175169,type,4635056,directed
26377582,1862626,type,470273,directed


In [9]:
df = relationship
df['start_type'] = relationship[':START_ID']
df['end_type'] = relationship[':END_ID']

In [10]:
df

,:START_ID,:TYPE,:END_ID,inverse,start_type,end_type
0,362633,birthPlace,5533231,direct,362633,5533231
1,362633,birthPlace,1056076,direct,362633,1056076
2,362633,deathPlace,5348598,direct,362633,5348598
3,362633,deathPlace,3773633,direct,362633,3773633
4,1324976,birthPlace,3429596,direct,1324976,3429596
...,...,...,...,...,...,...
26377579,1862626,type,4326933,directed,1862626,4326933
26377580,1862626,type,193423,directed,1862626,193423
26377581,5175169,type,4635056,directed,5175169,4635056
26377582,1862626,type,470273,directed,1862626,470273


In [8]:
type_dict = entity.set_index('entityId:ID')[':LABEL'].to_dict()

In [11]:
df['start_type'] = df['start_type'].map(type_dict)
df['end_type'] = df['end_type'].map(type_dict)

In [12]:
df

,:START_ID,:TYPE,:END_ID,inverse,start_type,end_type
0,362633,birthPlace,5533231,direct,Person,Place
1,362633,birthPlace,1056076,direct,Person,Place
2,362633,deathPlace,5348598,direct,Person,Place
3,362633,deathPlace,3773633,direct,Person,Place
4,1324976,birthPlace,3429596,direct,Person,Place
...,...,...,...,...,...,...
26377579,1862626,type,4326933,directed,Place,Person
26377580,1862626,type,193423,directed,Place,Person
26377581,5175169,type,4635056,directed,Place,Organisation
26377582,1862626,type,470273,directed,Place,Person


In [14]:
df['COUNTER'] =1
df

,:START_ID,:TYPE,:END_ID,inverse,start_type,end_type,COUNTER
0,362633,birthPlace,5533231,direct,Person,Place,1
1,362633,birthPlace,1056076,direct,Person,Place,1
2,362633,deathPlace,5348598,direct,Person,Place,1
3,362633,deathPlace,3773633,direct,Person,Place,1
4,1324976,birthPlace,3429596,direct,Person,Place,1
...,...,...,...,...,...,...,...
26377579,1862626,type,4326933,directed,Place,Person,1
26377580,1862626,type,193423,directed,Place,Person,1
26377581,5175169,type,4635056,directed,Place,Organisation,1
26377582,1862626,type,470273,directed,Place,Person,1


In [15]:
group_data = df.groupby(['start_type','end_type'])['COUNTER'].sum() #sum function

In [16]:
group_data

start_type  end_type    
Event       Event             51176
            Organisation      48008
            Others             5015
            Person           132908
            Place             48146
                             ...   
Work        Person          1175704
            Place            123087
            Species             686
            TimePeriod          196
            Work            1017106
Name: COUNTER, Length: 63, dtype: int64

In [22]:
chord = []
for i,j in group_data.index:
    temp = []
    temp.append(i)
    temp.append(j)
    temp.append(group_data[i][j])
    chord.append(temp)

In [23]:
chord

[['Event', 'Event', 51176],
 ['Event', 'Organisation', 48008],
 ['Event', 'Others', 5015],
 ['Event', 'Person', 132908],
 ['Event', 'Place', 48146],
 ['Event', 'Species', 13],
 ['Event', 'TimePeriod', 92],
 ['Event', 'Work', 1216],
 ['Organisation', 'Event', 48008],
 ['Organisation', 'Organisation', 501908],
 ['Organisation', 'Others', 169353],
 ['Organisation', 'Person', 1533144],
 ['Organisation', 'Place', 584568],
 ['Organisation', 'Species', 1016],
 ['Organisation', 'TimePeriod', 846724],
 ['Organisation', 'Work', 499153],
 ['Others', 'Event', 5015],
 ['Others', 'Organisation', 169353],
 ['Others', 'Others', 194020],
 ['Others', 'Person', 344508],
 ['Others', 'Place', 106866],
 ['Others', 'Species', 2782],
 ['Others', 'TimePeriod', 356],
 ['Others', 'Work', 382517],
 ['Person', 'Event', 132908],
 ['Person', 'Organisation', 1533144],
 ['Person', 'Others', 344508],
 ['Person', 'Person', 1264716],
 ['Person', 'Place', 2066443],
 ['Person', 'Species', 57055],
 ['Person', 'TimePeriod', 